# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv('output_data/cities.csv')

vacation_df.head()

,Unnamed: 0,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,0,Cabedelo,75,BR,1579917004,88,-6.98,-34.83,78.80,4.70
1,1,Punta Arenas,20,CL,1579916809,76,-53.15,-70.92,48.20,10.38
2,2,Severo-Kuril'sk,95,RU,1579917005,90,50.68,156.12,17.17,41.03
3,3,Butaritari,100,KI,1579917005,71,3.07,172.79,84.51,3.96
4,4,Fort Nelson,90,CA,1579917005,85,58.81,-122.70,10.40,2.24


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)


In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = vacation_df[["Latitude", "Longitude"]].astype(float)


humidity = vacation_df["Humidity"].astype(float)



In [15]:
 # Create a poverty Heatmap layer
fig = gmaps.figure(zoom_level=2,center=(0,0))


heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:

not_too_hot = vacation_df

not_too_hot.drop(not_too_hot.loc[not_too_hot['Max Temperature'] > 80].index, inplace=True)

print(len(not_too_hot))


11


In [17]:
not_too_cool = not_too_hot

not_too_cool.drop(not_too_cool.loc[not_too_cool['Max Temperature'] < 72].index, inplace=True)

print(len(not_too_cool))

11


In [8]:
not_too_humid = not_too_cool

not_too_humid.drop(not_too_humid.loc[not_too_humid['Humidity'] > 50].index, inplace=True)

not_too_humid.head(15)

,Unnamed: 0,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
12,12,Teloloapan,0,MX,1579917006,39,18.35,-99.85,73.99,3.11
20,20,Hobart,100,AU,1579916867,37,-42.88,147.33,73.99,21.92
37,37,New Norfolk,100,AU,1579917010,37,-42.78,147.06,73.99,21.92
83,83,Tessalit,85,ML,1579917018,12,20.20,1.01,72.77,12.50
113,113,Taoudenni,89,ML,1579917024,20,22.68,-3.98,72.52,5.88
148,148,Araouane,100,ML,1579917030,15,18.90,-3.53,73.92,6.51
326,326,Kita,83,ML,1579917061,10,13.03,-9.49,73.58,4.59
339,339,Jinka,100,ET,1579917063,48,5.65,36.65,72.21,4.23
525,525,Port Lincoln,0,AU,1579917095,48,-34.73,135.87,75.43,3.85
544,544,Adrar,99,MR,1579917098,39,20.50,-10.07,72.84,11.10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
hotel_df = not_too_humid

hotel_df['Hotel Name']=""

hotel_df.head(15)

,Unnamed: 0,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Hotel Name
12,12,Teloloapan,0,MX,1579917006,39,18.35,-99.85,73.99,3.11,
20,20,Hobart,100,AU,1579916867,37,-42.88,147.33,73.99,21.92,
37,37,New Norfolk,100,AU,1579917010,37,-42.78,147.06,73.99,21.92,
83,83,Tessalit,85,ML,1579917018,12,20.20,1.01,72.77,12.50,
113,113,Taoudenni,89,ML,1579917024,20,22.68,-3.98,72.52,5.88,
148,148,Araouane,100,ML,1579917030,15,18.90,-3.53,73.92,6.51,
326,326,Kita,83,ML,1579917061,10,13.03,-9.49,73.58,4.59,
339,339,Jinka,100,ET,1579917063,48,5.65,36.65,72.21,4.23,
525,525,Port Lincoln,0,AU,1579917095,48,-34.73,135.87,75.43,3.85,
544,544,Adrar,99,MR,1579917098,39,20.50,-10.07,72.84,11.10,


In [23]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    hotel_name = hotel_name.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        #cities_pd.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        #cities_pd.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [24]:
hotel_df.head(15)


,Unnamed: 0,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Hotel Name
12,12,Teloloapan,0,MX,1579917006,39,18.35,-99.85,73.99,3.11,Teloloapan
20,20,Hobart,100,AU,1579916867,37,-42.88,147.33,73.99,21.92,Hobart
37,37,New Norfolk,100,AU,1579917010,37,-42.78,147.06,73.99,21.92,New Norfolk
83,83,Tessalit,85,ML,1579917018,12,20.20,1.01,72.77,12.50,Achemelmel
113,113,Taoudenni,89,ML,1579917024,20,22.68,-3.98,72.52,5.88,Taoudenni
148,148,Araouane,100,ML,1579917030,15,18.90,-3.53,73.92,6.51,Araouane
326,326,Kita,83,ML,1579917061,10,13.03,-9.49,73.58,4.59,Kita
339,339,Jinka,100,ET,1579917063,48,5.65,36.65,72.21,4.23,K'Ak'o
525,525,Port Lincoln,0,AU,1579917095,48,-34.73,135.87,75.43,3.85,Port Lincoln
544,544,Adrar,99,MR,1579917098,39,20.50,-10.07,72.84,11.10,


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]  #narrowed_city_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [28]:
# Add marker layer ontop of heat map

fig.add_layer(locations)

fig

# Display Map

TraitError: Element of the 'layers' trait of a Map instance must be a Widget, but a value of      Latitude  Longitude
12      18.35     -99.85
20     -42.88     147.33
37     -42.78     147.06
83      20.20       1.01
113     22.68      -3.98
148     18.90      -3.53
326     13.03      -9.49
339      5.65      36.65
525    -34.73     135.87
544     20.50     -10.07
576      4.77      33.59 <class 'pandas.core.frame.DataFrame'> was specified.